# RawBased sin normalizar y sin cálculo de matrices de distancias

In [1]:
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
my_path = os.path.abspath('')
my_path = my_path.split('\\')
my_path_py = "\\".join(my_path[:-1])

DF_TOTAL_T=pd.read_csv(my_path_py+'\\DatosRaw\\DEN_2015to2020.csv', sep=',')

In [3]:
listaDistrito = DF_TOTAL_T['Dep-Prov-Distrito'].tolist()
listaDistrito = list(dict.fromkeys(listaDistrito))
print('Son ', len(listaDistrito), ' distritos')
listaDistrito.sort()
#print(listaDistrito)

Son  505  distritos


In [4]:
timeSeries = pd.read_csv(my_path_py+'\\DatosRaw\\SerieTemporal_Casos_2015to2020.csv', sep=',')
timeSeries = timeSeries.fillna(0.00001)

In [5]:
timeSeries

,0,1,2,3,4,5,6,7,8,9,...,308,309,310,311,312,313,314,315,316,317
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001
1,1.0,1.0,1.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.00000,7.00000,6.00000,10.00000,1.00000,4.00000,3.00000,9.00000,4.00000,1.00000
2,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001
501,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
502,1.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001
503,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


## Funciones de Distancias

In [6]:
import math
from math import sqrt, log, floor
from sklearn.metrics import mean_squared_error
from statistics import mean
from fastdtw import fastdtw
from scipy import stats
from scipy.spatial.distance import pdist

#Euclidean
def euclidean(x, y):
    r=np.linalg.norm(x-y)
    if math.isnan(r):
        r=1
    #print(r)
    return r

#Fast Dynamic time warping
def fast_DTW(x, y):
    r, _ = fastdtw(x, y, dist=euclidean)
    if math.isnan(r):
        r=1
    #print(r)
    return r

#Spearman
def scorr(x, y):
    r = stats.spearmanr(x, y)[0]
    if math.isnan(r):
        r=0
    #print(r)
    return 1 - r

#RMSE
def rmse(x, y):
    r=sqrt(mean_squared_error(x,y))
    if math.isnan(r):
        r=1
    #print(r)
    return r

def lcs(a, b):  
    lengths = [[0 for j in range(len(b)+1)] for i in range(len(a)+1)]
    # row 0 and column 0 are initialized to 0 already
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            if x == y:
                lengths[i+1][j+1] = lengths[i][j] + 1
            else:
                lengths[i+1][j+1] = max(lengths[i+1][j], lengths[i][j+1])
    x, y = len(a), len(b)
    result = lengths[x][y]
    return result

def discretise(x):
    return int(x * 10)

def multidim_lcs(a, b):
    a = a.applymap(discretise)
    b = b.applymap(discretise)
    rows, dims = a.shape
    lcss = [lcs(a[i+2], b[i+2]) for i in range(dims)]
    return 1 - sum(lcss) / (rows * dims)

#Correlation
def corr(x, y):
    r=np.dot(x-mean(x),y-mean(y))/((np.linalg.norm(x-mean(x)))*(np.linalg.norm(y-mean(y))))
    if math.isnan(r):
        r=0
    #print(r)
    return 1 - r

In [7]:
n= timeSeries.shape[0]
timeSeries = timeSeries.replace('nan', np.nan).fillna(0.000001)
timeSeries = timeSeries.replace([np.inf, -np.inf], np.nan).fillna(0.000001)

In [8]:
timeSeries.shape

(505, 318)

## Matriz de distancia euclidiana

In [9]:
#Euclidean
euclidean_dist = pd.read_csv('MatrizEuclidiana_RB.csv', sep=',').to_numpy()

## Matriz de DTW 

In [10]:
#DTW 
dtw_dist = pd.read_csv('MatrizDTW_RB.csv', sep=',').to_numpy()

## Matriz de correlación de Pearson

In [11]:
corr_dist = pd.read_csv('MatrizPearson_RB.csv', sep=',').to_numpy()

## Matriz de correlación de Spearman

In [12]:
#scorr
scorr_dist = pd.read_csv('MatrizSpearman_RB.csv', sep=',').to_numpy()

## DEFINICIÓN DEL NÚMERO DE CLUSTERS K

In [13]:
k=6

In [14]:
from sklearn.preprocessing import LabelEncoder
DIAMETER_METHODS = ['mean_cluster', 'farthest']
CLUSTER_DISTANCE_METHODS = ['nearest', 'farthest']

def inter_cluster_distances(labels, distances, method='nearest'):
    """Calculates the distances between the two nearest points of each cluster.
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param method: `nearest` for the distances between the two nearest points in each cluster, or `farthest`
    """
    if method not in CLUSTER_DISTANCE_METHODS:
        raise ValueError(
            'method must be one of {}'.format(CLUSTER_DISTANCE_METHODS))

    if method == 'nearest':
        return __cluster_distances_by_points(labels, distances)
    elif method == 'farthest':
        return __cluster_distances_by_points(labels, distances, farthest=True)


def __cluster_distances_by_points(labels, distances, farthest=False):
    n_unique_labels = len(np.unique(labels))
    cluster_distances = np.full((n_unique_labels, n_unique_labels),
                                float('inf') if not farthest else 0)

    np.fill_diagonal(cluster_distances, 0)

    for i in np.arange(0, len(labels) - 1):
        for ii in np.arange(i, len(labels)):
            if labels[i] != labels[ii] and (
                (not farthest and
                 distances[i, ii] < cluster_distances[labels[i], labels[ii]])
                    or
                (farthest and
                 distances[i, ii] > cluster_distances[labels[i], labels[ii]])):
                cluster_distances[labels[i], labels[ii]] = cluster_distances[
                    labels[ii], labels[i]] = distances[i, ii]
    return cluster_distances


def diameter(labels, distances, method='farthest'):
    """Calculates cluster diameters
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param method: either `mean_cluster` for the mean distance between all elements in each cluster, or `farthest` for the distance between the two points furthest from each other
    """
    if method not in DIAMETER_METHODS:
        raise ValueError('method must be one of {}'.format(DIAMETER_METHODS))

    n_clusters = len(np.unique(labels))
    diameters = np.zeros(n_clusters)

    if method == 'mean_cluster':
        for i in range(0, len(labels) - 1):
            for ii in range(i + 1, len(labels)):
                if labels[i] == labels[ii]:
                    diameters[labels[i]] += distances[i, ii]

        for i in range(len(diameters)):
            diameters[i] /= sum(labels == i)

    elif method == 'farthest':
        for i in range(0, len(labels) - 1):
            for ii in range(i + 1, len(labels)):
                if labels[i] == labels[ii] and distances[i, ii] > diameters[
                        labels[i]]:
                    diameters[labels[i]] = distances[i, ii]
    return diameters

def dunn(labels, distances, diameter_method='farthest',
         cdist_method='nearest'):
    """
    Dunn index for cluster validation (larger is better).
    
    .. math:: D = \\min_{i = 1 \\ldots n_c; j = i + 1\ldots n_c} \\left\\lbrace \\frac{d \\left( c_i,c_j \\right)}{\\max_{k = 1 \\ldots n_c} \\left(diam \\left(c_k \\right) \\right)} \\right\\rbrace
    
    where :math:`d(c_i,c_j)` represents the distance between
    clusters :math:`c_i` and :math:`c_j`, and :math:`diam(c_k)` is the diameter of cluster :math:`c_k`.
    Inter-cluster distance can be defined in many ways, such as the distance between cluster centroids or between their closest elements. Cluster diameter can be defined as the mean distance between all elements in the cluster, between all elements to the cluster centroid, or as the distance between the two furthest elements.
    The higher the value of the resulting Dunn index, the better the clustering
    result is considered, since higher values indicate that clusters are
    compact (small :math:`diam(c_k)`) and far apart (large :math:`d \\left( c_i,c_j \\right)`).
    :param labels: a list containing cluster labels for each of the n elements
    :param distances: an n x n numpy.array containing the pairwise distances between elements
    :param diameter_method: see :py:function:`diameter` `method` parameter
    :param cdist_method: see :py:function:`diameter` `method` parameter
    
    .. [Kovacs2005] Kovács, F., Legány, C., & Babos, A. (2005). Cluster validity measurement techniques. 6th International Symposium of Hungarian Researchers on Computational Intelligence.
    """

    labels = LabelEncoder().fit(labels).transform(labels)
    
    

    ic_distances = inter_cluster_distances(labels, distances, cdist_method)
    #print("IC",ic_distances)
    if len(ic_distances[ic_distances.nonzero()])==0:
        min_distance = 0
    else:
        min_distance = min(ic_distances[ic_distances.nonzero()])
    max_diameter = max(diameter(labels, distances, diameter_method))
    
    

    return min_distance / max_diameter

In [16]:
import scipy.cluster.hierarchy as hac
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
SIL =[]
CHZ_ =[]
DUNN_ =[]
DAVIES_ =[]

#Experimentos HAC
HAC_euc = AgglomerativeClustering(n_clusters=k).fit_predict(euclidean_dist)
print("HAC + euclidian distance: ")
sil = silhouette_score(euclidean_dist, HAC_euc)
cal= calinski_harabasz_score(euclidean_dist, HAC_euc)
davies_ = davies_bouldin_score(euclidean_dist, HAC_euc)
try:
    dunn_ = dunn(HAC_euc, euclidean_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)

SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

print("-------------------")

HAC_corr = AgglomerativeClustering(n_clusters=k).fit_predict(corr_dist)
print("HAC + corr distance: ")
sil = silhouette_score(corr_dist, HAC_corr)
cal= calinski_harabasz_score(corr_dist, HAC_corr)
davies_ = davies_bouldin_score(corr_dist, HAC_corr)
try:
    dunn_ = dunn(HAC_corr, corr_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)
SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

print("-------------------")

HAC_scorr = AgglomerativeClustering(n_clusters=k).fit_predict(scorr_dist)
print("HAC + scorr distance: ")
sil = silhouette_score(scorr_dist, HAC_scorr)
cal= calinski_harabasz_score(scorr_dist, HAC_scorr)
davies_ = davies_bouldin_score(scorr_dist, HAC_scorr)
try:
    dunn_ = dunn(HAC_scorr, scorr_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)
SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

print("-------------------")

HAC_dtw = AgglomerativeClustering(n_clusters=k).fit_predict(dtw_dist)
print("HAC + dtw distance: ")
sil = silhouette_score(dtw_dist, HAC_dtw)
cal= calinski_harabasz_score(dtw_dist, HAC_dtw)
davies_ = davies_bouldin_score(dtw_dist, HAC_dtw)
try:
    dunn_ = dunn(HAC_dtw, dtw_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)
SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\cluster\hierarchy.py:826: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  return linkage(y, method='ward', metric='euclidean')


HAC + euclidian distance: 
SC:  0.7555016941885081
CHZ:  2149.2529957382853
DUNN:  0.11975131490841735
Davies:  0.5770749020222979
-------------------
HAC + corr distance: 
SC:  0.6190350105023912
CHZ:  619.1839505561219
DUNN:  0.9073348158248457
Davies:  0.8417970647671499
-------------------
HAC + scorr distance: 
SC:  0.6126581791919001
CHZ:  525.5057120536561
DUNN:  0.6536554202840612
Davies:  1.07069044045481
-------------------
HAC + dtw distance: 
SC:  0.6555852427676193
CHZ:  1026.8588895779778
DUNN:  0.16831485155576842
Davies:  0.8281366236479585


In [17]:
from sklearn.cluster import KMeans, DBSCAN

km_euc = KMeans(n_clusters=k).fit_predict(euclidean_dist)
silhouette_avg=silhouette_score(euclidean_dist, km_euc)
print("KM + euclidian distance: ")
sil = silhouette_score(euclidean_dist, km_euc)
cal= calinski_harabasz_score(euclidean_dist, km_euc)
davies_ = davies_bouldin_score(euclidean_dist, km_euc)
try:
    dunn_ = dunn(km_euc, euclidean_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)

SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

print("-------------------------")

km_corr = KMeans(n_clusters=k).fit_predict(corr_dist)
print("KM + corr distance: ")
sil = silhouette_score(corr_dist, km_corr)
cal= calinski_harabasz_score(corr_dist, km_corr)
davies_ = davies_bouldin_score(corr_dist, km_corr)
try:
    dunn_ = dunn(km_corr, corr_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)
SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

print("-------------------------")

km_scorr = KMeans(n_clusters=k).fit_predict(scorr_dist)
print("KM + scorr distance: ")
HAC_scorr = AgglomerativeClustering(n_clusters=k).fit_predict(scorr_dist)
print("HAC + scorr distance: ")
sil = silhouette_score(scorr_dist, km_scorr)
cal= calinski_harabasz_score(scorr_dist, km_scorr)
davies_ = davies_bouldin_score(scorr_dist, km_scorr)
try:
    dunn_ = dunn(km_scorr, scorr_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)
SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

print("-------------------------")

km_dtw = KMeans(n_clusters=k).fit_predict(dtw_dist)
print("KM + dtw distance: ")
sil = silhouette_score(dtw_dist, km_dtw)
cal= calinski_harabasz_score(dtw_dist, km_dtw)
davies_ = davies_bouldin_score(dtw_dist, km_dtw)
try:
    dunn_ = dunn(km_dtw, dtw_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)
SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

KM + euclidian distance: 
SC:  0.7557470937069032
CHZ:  2309.263534551376
DUNN:  0.08672637194806244
Davies:  0.583980416750956
-------------------------
KM + corr distance: 
SC:  0.6456727940230521
CHZ:  680.4616441978287
DUNN:  0.9073348158248457
Davies:  0.8259558808091633
-------------------------
KM + scorr distance: 
HAC + scorr distance: 
SC:  0.6425007187825584
CHZ:  629.7328450128024
DUNN:  0.6598149279470464
Davies:  0.9218236134012923
-------------------------
KM + dtw distance: 
SC:  0.6819160897430984
CHZ:  1178.5735016005247
DUNN:  0.16592498550933762
Davies:  0.8098672437092652


# Experimentos DBSCAN

El puntaje es negativo porque los puntos están en promedio más cerca de otro cluster, que del cuál ha sido asignado

In [ ]:
epsilon_lista = np.arange (0.01, 4, 0.01)
sil_list = []
for e in epsilon_lista:
    try:
        DB_corr = DBSCAN(eps=e, min_samples=5).fit_predict(corr_dist)
        sil = silhouette_score(corr_dist, DB_corr)
        sil_list.append(sil)
    except:
        pass
plt.plot(sil_list)
print('Epsilon máximo: ', epsilon_lista[np.argmax(sil_list)])

In [18]:
DB_euc = DBSCAN(eps=220, min_samples=3).fit_predict(euclidean_dist)
print("DBScan + euclidian distance: ")
sil = silhouette_score(euclidean_dist, DB_euc)
cal= calinski_harabasz_score(euclidean_dist, DB_euc)
davies_ = davies_bouldin_score(euclidean_dist, DB_euc)
try:
    dunn_ = dunn(DB_euc, euclidean_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)

SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

print("---------------------------")

#CON CORRELATION
DB_corr = DBSCAN(eps=1.61, min_samples=3).fit_predict(corr_dist)
print("DBSCAN + corr distance: ")
sil = silhouette_score(corr_dist, DB_corr)
cal= calinski_harabasz_score(corr_dist, DB_corr)
davies_ = davies_bouldin_score(corr_dist, DB_corr)
try:
    dunn_ = dunn(DB_corr, corr_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)
SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

print("---------------------------")

#CON SPEARMAN
DB_scorr = DBSCAN(eps=1.61, min_samples=3).fit_predict(scorr_dist)
print("DBSCAN + scorr distance: ")
sil = silhouette_score(scorr_dist, DB_scorr)
cal= calinski_harabasz_score(scorr_dist, DB_scorr)
davies_ = davies_bouldin_score(scorr_dist, DB_scorr)
try:
    dunn_ = dunn(DB_scorr, scorr_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)
SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)
print("---------------------------")

#CON D TIME WARPING
DB_dtw = DBSCAN(eps=220, min_samples=5).fit_predict(dtw_dist)
print("DBSCAN + dtw distance: ")
sil = silhouette_score(dtw_dist, DB_dtw)
cal= calinski_harabasz_score(dtw_dist, DB_dtw)
davies_ = davies_bouldin_score(dtw_dist, DB_dtw)
try:
    dunn_ = dunn(DB_dtw, dtw_dist, 'farthest', 'farthest')
except:
    dunn_ = np.nan
print("SC: ", sil)
print("CHZ: ", cal)
print("DUNN: ", dunn_)
print("Davies: ", davies_)

SIL.append(sil)
CHZ_.append(cal)
DUNN_.append(dunn_)
DAVIES_.append(davies_)

DBScan + euclidian distance: 
SC:  0.4363244592044793
CHZ:  76.7500119068537
DUNN:  0.03158907587512208
Davies:  0.9179159240166097
---------------------------
DBSCAN + corr distance: 
SC:  0.5759711275056483
CHZ:  211.11032606123806
DUNN:  0.9073348158248457
Davies:  1.18497296190995
---------------------------
DBSCAN + scorr distance: 
SC:  0.5012463692160323
CHZ:  116.54561489053813
DUNN:  0.5004913103340635
Davies:  1.1216694940326069
---------------------------
DBSCAN + dtw distance: 
SC:  0.27347640108948756
CHZ:  53.83544045872255
DUNN:  0.0009625094847589598
Davies:  1.1073656398681977


In [19]:
aux2 = pd.DataFrame()
aux2['Dep-Prov-Distrito'] =listaDistrito
aux2['Cluster KM Eu'] = km_euc
aux2['Cluster HAC Eu'] = HAC_euc
aux2['Cluster DB Pear'] = DB_corr
aux2.to_csv('ClusterRBCasos_Peru.csv', index = False)

In [20]:
np.unique(DB_corr)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7], dtype=int64)

In [23]:
list_m = []    
list_m.append('Euclidean')
list_m.append('Pearson')
list_m.append('Spearman')
list_m.append('DTW')

df_sc = pd.DataFrame()   
df_sc['Metrica'] = list_m

df_sc['SIL HAC RB'] = [SIL[0], SIL[1], SIL[2], SIL[3]]
df_sc['SIL KM RB'] = [SIL[4], SIL[5], SIL[6], SIL[7]]
df_sc['SIL DB RB'] = [SIL[8], SIL[9], SIL[10], SIL[11]]

df_sc['CHZ HAC RB'] = [CHZ_[0], CHZ_[1], CHZ_[2], CHZ_[3]]
df_sc['CHZ KM RB'] = [CHZ_[4], CHZ_[5], CHZ_[6], CHZ_[7]]
df_sc['CHZ DB RB'] = [CHZ_[8], CHZ_[9], CHZ_[10], CHZ_[11]]


df_sc['DUNN HAC RB'] = [DUNN_[0], DUNN_[1], DUNN_[2], DUNN_[3]]
df_sc['DUNN KM RB'] = [DUNN_[4], DUNN_[5], DUNN_[6], DUNN_[7]]
df_sc['DUNN DB RB'] = [DUNN_[8], DUNN_[9], DUNN_[10], DUNN_[11]]

df_sc['DAVIES HAC RB'] = [DAVIES_[0], DAVIES_[1], DAVIES_[2], DAVIES_[3]]
df_sc['DAVIES KM RB'] = [DAVIES_[4], DAVIES_[5], DAVIES_[6], DAVIES_[7]]
df_sc['DAVIES DB RB'] = [DAVIES_[8], DAVIES_[9], DAVIES_[10], DAVIES_[11]]


df_sc

,Metrica,SIL HAC RB,SIL KM RB,SIL DB RB,CHZ HAC RB,CHZ KM RB,CHZ DB RB,DUNN HAC RB,DUNN KM RB,DUNN DB RB,DAVIES HAC RB,DAVIES KM RB,DAVIES DB RB
0,Euclidean,0.755502,0.755747,0.436324,2149.252996,2309.263535,76.750012,0.119751,0.086726,0.031589,0.577075,0.583980,0.917916
1,Pearson,0.619035,0.645673,0.575971,619.183951,680.461644,211.110326,0.907335,0.907335,0.907335,0.841797,0.825956,1.184973
2,Spearman,0.612658,0.642501,0.501246,525.505712,629.732845,116.545615,0.653655,0.659815,0.500491,1.070690,0.921824,1.121669
3,DTW,0.655585,0.681916,0.273476,1026.858890,1178.573502,53.835440,0.168315,0.165925,0.000963,0.828137,0.809867,1.107366


In [24]:
df_sc.to_csv('SilScoreRBCasos_2015to2020.csv', index=False)